# <center> Creating a list of occupations 

To recognise occupations in the captions, a comprehensive list of all possible occupations must be constructed. The difficulty with this task is that official job titles that one may find online are too specific and do not correspond to occupations people tend to mention when speaking (or writing captions). Therefore, the constructed list of occupations has to contain both generic terms but also specific occupations. To do this, the following approach was taken:
1. Five datasets of varying size and specificity were loaded. In total, these add up to around 33,000 occupations.
 - [**O*NET**](https://www.onetonline.org/find/all)  
 - [**ESCO (ESCO dataset v1.2.0)**](https://esco.ec.europa.eu/en/use-esco/download)   
 - [**Kaggle Job Description Dataset**](https://www.kaggle.com/datasets/ravindrasinghrana/job-description-dataset)  
 - [**US Labor Statistics (May 2024, all data)** ](https://www.bls.gov/oes/tables.htm)   
 - [**US Census Data (2018 Census Occupation Index)**](https://www.census.gov/topics/employment/industry-occupation/guidance/indexes.html)
2. Each dataset is cleaned with the same function, included in the cell below. The steps taken for the cleaning are:
 - Make everything lowercase
 - split jobs at "and" in a way that "sales and marketing manager" becomes "sales manager" and "marketing manager".
 - split jobs of the form "truck, ship, and boat driver" becomes "truck driver", "ship driver" and "boat driver". 
 - Other commmon cases such as this are also treated, see the function definition. There is still room for improvement in this aspect.
 - If there is a comma at the end, and it is followed by 2 or 1 words, remove the comma and the words.
 - Drop everything including "in" from the text
 - removing duplicates
3. The list is enriched by adding a synonyms column which contains the occupation and its plural form. The idea was to enrich this list by using `spacy.load('en_core_web_sm')`. This however introduced synonyms that are not actual occupations "valet <-> gentleman". 
4. Around 1000 extra words are added 
5. The constructed list is saved as a `csv` file: `final_combined_occupations.csv`.

<div style="border: 1px solid #f5c518; background-color: #fff8e1; padding: 10px; border-radius: 5px;">
<b>Note:</b> This file has very specific versions of packages to ensure compatibility. 
Do not run the notebook in an environment with different package versions, as it may not work.
</div>


In [1]:
#Loading packages 

# Data manipulation
import numpy as np
import pandas as pd


# Language processing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.matcher import PhraseMatcher
from collections import Counter




nltk.download('punkt')       # Tokeniser
nltk.download('stopwords')   # Stopwords list
nltk.download('wordnet')     # Lemmatiser
nlp = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = WordNetLemmatizer() # Initialise lemmatiser
occupations = pd.read_csv("All_Occupations.csv")

[nltk_data] Downloading package punkt to C:\Users\andra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\andra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


---
## Dataset from [**O*NET**](https://www.onetonline.org/find/all)

This dataset has has three columns, one for the area of the job (what level of qualification we need for the job), a code (SOC code) which allows the categorisation of the jobs (if needed) according to that described on their site.

I will only really need the list of occupations. The name of occupations here are quite general and I only need to split up words like "accountants and auditors" into two separate rows: "accountants" and "auditors". Thi is all done in the cell below, and saved to a new (redundant) dataset. Then, duplicates are removed and everything is lowercased. The type of each entry is a string.

In [2]:
#split "and" into two separate occupations
def split_and_expand(text):
    parts = text.split(" and ")
    if len(parts) != 2:
        return [text.strip()]  # Return the original text if "and" not found
    left_part = parts[0].strip()
    right_part = parts[1].strip()
    
    # Expand left part
    left_expanded = []
    if ',' in left_part:
        left_subparts = [subpart.strip() for subpart in left_part.split(',')]
        for subpart in left_subparts:
            left_expanded.append(f"{subpart} {right_part}".strip())
    else:
        left_expanded.append(left_part)
    
    # Expand right part
    right_expanded = []
    if ',' in right_part:
        right_subparts = [subpart.strip() for subpart in right_part.split(',')]
        for subpart in right_subparts:
            right_expanded.append(f"{left_part} {subpart}".strip())
    else:
        right_expanded.append(right_part)
    return [left_expanded, right_expanded]

# Apply the function to each row → produces lists
occupations["Occupation expanded"] = occupations['Occupation'].apply(split_and_expand)

# Explode lists into separate rows
occupations = occupations.explode("Occupation expanded", ignore_index=True)
occupations = occupations.rename(columns={"Occupation": "Occupation_drop"})
occupations = occupations.rename(columns={"Occupation expanded": "Occupation"})

#drop Occupation_drop
occupations = occupations.drop(columns=['Occupation_drop'])

#elements that are lists should just be strings
occupations['Occupation'] = occupations['Occupation'].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)

#make lower string
occupations['Occupation'] = occupations['Occupation'].str.lower()

#remove duplicates
occupations = occupations.drop_duplicates(subset=['Occupation'], ignore_index=True)

#drop Data-level columns
occupations = occupations.drop(columns=['Data-level'])

print(occupations)
occupations.to_csv("occupations1.csv", index=False)

      Job Zone        Code           Occupation
0          4.0  13-2011.00          accountants
1          4.0  13-2011.00             auditors
2          2.0  27-2011.00               actors
3          4.0  15-2011.00            actuaries
4          5.0  29-1291.00       acupuncturists
...        ...         ...                  ...
1159       2.0  43-9022.00              typists
1160       4.0  27-3043.00              writers
1161       4.0  27-3043.00              authors
1162       4.0  19-1023.00           zoologists
1163       4.0  19-1023.00  wildlife biologists

[1164 rows x 3 columns]


To expand the list of occupations created by the previous cell, I count the most common word occurences in the job titles. For example, "manager" occurs frequently, and so do "technicians" and such. Since I want my list of occupations to be general and specific, I will save the most common word occurences that are also occupations. These will be later added to my final datbase. 

This is all done with counter and nltk packages, and I once again save it to a csv file, to avoid having to re-run code (even if in the end the code runs quite fast.)

In [3]:
#find common occurences of words to generate "general" occupations

occupations1 = pd.read_csv("occupations1.csv")
occupation1 = occupations1.dropna(subset=['Occupation']).reset_index(drop=True)
all_words = ' '.join(occupation1['Occupation']).split()
word_counts = Counter(all_words)
common_words = word_counts.most_common(1000)
print(common_words)

#remove stopwords from most common words
common_words = [(word, count) for word, count in common_words if word.lower() not in stop_words]
print(common_words)

#save most common words to csv
common_words_df = pd.DataFrame(common_words, columns=['Word', 'Count'])
common_words_df.to_csv("most_common_words.csv", index=False)

[('and', 143), ('other', 58), ('all', 52), ('technicians', 51), ('workers', 49), ('managers', 48), ('operators', 40), ('specialists', 39), ('engineers', 39), ('machine', 38), ('teachers,', 37), ('operators,', 29), ('of', 26), ('postsecondary', 26), ('equipment', 25), ('technologists', 24), ('education', 22), ('workers,', 22), ('supervisors', 22), ('setters,', 22), ('clerks', 21), ('first-line', 21), ('officers', 19), ('service', 19), ('except', 19), ('installers', 18), ('repairers', 18), ('metal', 18), ('tenders', 17), ('analysts', 17), ('assistants', 16), ('computer', 15), ('medical', 15), ('plastic', 14), ('special', 14), ('sales', 13), ('engineering', 13), ('scientists', 13), ('food', 13), ('tenders,', 13), ('attendants', 12), ('health', 12), ('science', 11), ('therapists', 10), ('school', 10), ('agents', 9), ('inspectors', 9), ('mechanics', 9), ('systems', 9), ('plant', 9), ('social', 9), ('information', 9), ('transportation', 9), ('financial', 9), ('gambling', 9), ('operations', 8

Here is the list that was created with my method, after I gave it to ChatGPT to filter out all that are not actual occupations.

In [4]:
occupations_1 = [
    "technicians","workers","managers","operators","specialists","engineers","teachers","technologists","supervisors","clerks","officers","installers",
    "repairers","analysts","assistants","attendants","scientists","sales","therapists","agents","inspectors","mechanics","designers","assemblers","physicians",
    "directors","counselors","aides","nurses","drivers","planners","repairers","cooks","assistants","scientists","truck drivers","cutters","cleaners","nurse",
    "coordinators","architects","representatives","administrators","secretaries","police","writers","occupational therapists","instructors","trainers",
    "drafters","artists","biologists","investigators","psychologists","administrators","dental assistants","advisors","engineers","executive","firefighters",
    "housekeeping staff","retail workers","funeral directors","lawyers","librarians","marketing specialists","trimmers","ophthalmic technicians","pediatric nurses",
    "railroad workers","teachers","web developers", "auditors", "performers", "athletes", "controllers", "caretakers", "appraisers", "assessors", "collectors", 
    "brickmasons", "carpenters", "pilots", "cashiers", "neuropsychologists", "programmers", "laborers", "clerks", "detectives", "economists", "editors", 
    "epidemiologists", "fabricators", "surveyors", "practitioners", "maids", "midwives", "military", "nursing assistants", "surgeons", "salespersons", "testers", 
    "therapists", "physicians", "pathologists", "pumpers", "recycling workers", "robotics engineers", "representatives", "teachers", "software developers", 
    "speech-language pathologists", "tax preparers", "veterinary technicians", "accountants", "actors", "actuaries", "acupuncturists", "allergists", "immunologists", 
    "anesthesiologists", "breeders", "anthropologists", "archeologists", "archivists", "astronomers", "audiologists", "bailiffs", "bakers", "barbers", "baristas", 
    "bartenders", "bellhops", "biochemists", "biophysicists", "bioengineers", "biostatisticians", "boilermakers", "blockmasons", "announcers", "jockeys", "butchers", 
    "buyers", "cabinetmakers", "cardiologists", "cartographers", "photogrammetrists", "chefs", "cooks", "chemists", "chiropractors", "executives", "choreographers", 
    "clergy", "coaches", "scouts", "divers", "engineers", "architects", "concierges", "coroners", "jailers", "cost estimators", "counselors", "couriers", "messengers", 
    "court reporters", "captioners", "craft workers", "crane operators", "crematory operators", "curators", "customer service representatives", "cytotechnologists", 
    "dancers", "data entry keyers", "demonstrators", "promoters", "dental hygienists", "dermatologists", "designers", "publishers", "diagnostic sonographers", 
    "dietitians", "nutritionists", "dishwashers", "drafters", "driver/sales workers", "drywall installers", "electricians", "electromechanical technicians", 
    "elevator installers", "embalmers", "entertainers", "etchers", "engravers", "excavating operators", "physiologists", "patternmakers", "fallers", "educators", 
    "contractors", "farmworkers", "fashion designers", "fence erectors", "fiberglass laminators", "illustrators", "firefighters", "groundskeepers", "material-moving workers", 
    "specialists", "wardens", "fishers", "foresters", "foundry workers", "coremakers", "fuel cell technicians","fundraisers", "dealers", "geneticists", "geographers", 
    "glaziers", "cosmetologists", "registrars", "practitioners", "heating and air conditioning technicians", "tractor-trailer drivers", "helpers", "historians", 
    "histotechnologists", "hospitalists", "hosts", "hostesses", "ergonomists", "hydrologists", "ecologists", "industrial-organizational psychologists", "weighers", "appraisers", 
    "underwriters", "interior designers", "interpreters", "translators", "janitors", "jewelers", "magistrates", "movers", "landscapers", "lathe operators", "laundry workers", 
    "lawyers", "legislators", "librarians", "locksmiths", "logisticians", "feeders", "offbearers", "machinists", "manicurists", "pedicurists", "mathematicians", "programmers", 
    "directors", "managers", "dosimetrists", "transcriptionists", "merchandise displayers", "metal-refining workers", "microbiologists", "enlisted personnel", "officers", 
    "millwrights", "mechanics", "casting workers", "arrangers", "projectionists", "museum conservators", "composers","musical tuners", "musicians", "singers", "nannies", 
    "neurologists", "journalists", "anesthetists", "obstetricians", "gynecologists", "merchants", "ophthalmologists", "opticians", "optometrists", "orthodontists", 
    "orthotists", "orthoptists", "prosthetists", "pediatricians", "pharmacists", "phlebotomists", "photographers", "physicians", "physicists", "driver", "pipelayers", 
    "plasterers", "podiatrists", "sheriffs", "patrol officers", "carriers", "postmasters", "superintendents", "potters", "pourers", "distributors", "dispatchers", "producers", 
    "proofreaders", "prosthodontists", "psychiatrists", "psychologists", "president", "ceo", "cfo", "cto", "coo", "founder", "entrepreneur","astronaut", "cosmonaut", 
    "marine biologist", "zoologist", "geologist","paleontologist", "archaeologist", "environmental scientist", "climatologist","software architect", "data scientist", 
    "machine learning engineer", "ai researcher","quantitative analyst", "actuary", "urban planner", "civil engineer", "mechanical engineer","electrical engineer", 
    "chemical engineer", "nuclear engineer", "aerospace engineer","pilot", "air traffic controller", "ship captain", "sailor", "diplomat", "ambassador","senator", 
    "congressperson", "judge", "chief justice", "law enforcement officer","detective", "fire chief", "paramedic", "emergency medical technician","composer", 
    "conductor", "choreographer", "artist", "actor", "director", "producer","photographer", "journalist", "editor", "novelist", "screenwriter", "poet","philosopher", 
    "historian", "linguist", "psychologist", "psychiatrist","surgeon", "cardiologist", "oncologist", "neurologist", "radiologist", "dentist","veterinarian", "nurse practitioner", 
    "midwife", "therapist", "social worker","teacher", "professor", "lecturer", "researcher", "scientist"
]



This function was initially used to get synonyms to the occupations I found. However, often synonyms are not actual jobs, which leads to a lot of confusion when counting job occurences in text. For the time being, the function remains here, but I suggest the analysis to not use synonyms.

In [5]:
#synonyms for occupations
from nltk.corpus import wordnet as wn

def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word, pos=wn.NOUN):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))
    return synonyms

---
# External datasets


---
## [**ESCO (ESCO dataset v1.2.0)**](https://esco.ec.europa.eu/en/use-esco/download)

This dataset contains jobs from an EU commissions source. There are again many other fields in the dataset, but we are not interested in these. There are roughly 3,300 job entries in this dataset that I will be able to use.

The cleaning of this dataset will be done with a generic function that I will also apply later on to the other datasets. It deals exactly with what is above, with more specific cases about dealing with commas and "and":

1. If there is a comma at the end, and it is followed by 2 or 1 words, remove the comma and the words.
2. If there is a sentence with example1, example2, and example3 manager -> turn it into [example1 manager, example2 manager, example3 manager] and then explode the dataset
3. if there is example1 and example2 operator -> [example1 operator, example2 operator]
4. if there is care-taker of tables and chairs -> [care-taker of tables, care-taker of chairs]
5. Drop everything including "in" from the text
6. If there is something like fitter and turner -> [fitter, turner]
This all needs to be done to each of the datasets.

In [6]:
from pydoc import text


def preprocess_occupation(data):

    # Making everything lowercase
    data['Occupation'] = data['Occupation'].str.lower()

    # Removing duplicates (if any)
    data = data.drop_duplicates(subset=['Occupation'], ignore_index=True)
    
    # 1. Remove trailing comma followed by 1 or 2 words (engineer, all others -> engineer)
    data['Occupation'] = data['Occupation'].apply(
        lambda x: re.sub(r',\s*\b\w+\b(\s+\b\w+\b)?$', '', x)
    )

    # 2. Handle "example1, example2, and example3 manager" -> [example1 manager, example2 manager, example3 manager]
    def expand_commas(text):
            
        # Pattern: words separated by commas + "and" + final word(s)
        match = re.match(r'^(.*?),\s*and\s*(\w+)\s+(.*)$', text)
        if match:
            before, last_item, shared_tail = match.groups()
            parts = [p.strip() for p in before.split(',')]
            parts.append(last_item)
            return [f"{p} {shared_tail}" for p in parts]
        return [text]

    # 3. Handle "example1 and example2 operator" -> [example1 operator, example2 operator]
    def expand_and(text):
        match = re.match(r'(.+?)\s+and\s+(.+?)\s+(\w+)$', text)
        if match:
            return [match.group(1) + ' ' + match.group(3), match.group(2) + ' ' + match.group(3)]
        return [text]

    # 4. Handle "care-taker of tables and chairs" -> [care-taker of tables, care-taker of chairs]
    def expand_of_and(text):
        match = re.match(r'(.+? of) (.+?) and (.+)', text)
        if match:
            return [match.group(1) + ' ' + match.group(2), match.group(1) + ' ' + match.group(3)]
        return [text]
    
    # 5. Drop everything including "in" (specialist in something -> specialist)
    def drop_in(text):
        return text.split(' in ')[0].strip()

    # 6. Handle "fitter and turner" -> [fitter, turner]
    def split_simple_and(text):
        if re.match(r'^\w+\s+and\s+\w+$', text):
            return [t.strip() for t in text.split(' and ')]
        return [text]

    # Combine all expansion rules
    all_expanded = []
    for occ in data['Occupation']:
        temp = expand_commas(occ)
        temp2 = []
        for t in temp:
            temp2.extend(expand_and(t))
        temp3 = []
        for t in temp2:
            temp3.extend(expand_of_and(t))
        temp4 = []
        for t in temp3:
            t = drop_in(t)
            temp4.extend(split_simple_and(t))
        all_expanded.extend(temp4)

    # Create final DataFrame
    clean_data = pd.DataFrame({'Occupation': list(set(all_expanded))})
    clean_data = clean_data.sort_values('Occupation').reset_index(drop=True)

    return clean_data


Applying it to the EU dataset we see we have now a list of around 3100 occupations, that (should) all be unique.

In [7]:
# EU dataset
eu_occ = pd.read_csv("occupations_en.csv")
eu_occ = eu_occ[['preferredLabel']]
eu_occ.columns = ['Occupation']
eu_occ = preprocess_occupation(eu_occ)

print(f"There are {len(eu_occ)} occupations in the EU dataset.")
print(eu_occ.head())

There are 3053 occupations in the EU dataset.
                       Occupation
0                     3d animator
1                     3d modeller
2          3d printing technician
3      abrasive blasting operator
4  absorbent pad machine operator


However, there are still some case where "and" occurs but I have not fixed. I deem these too specific anyways, so I just drop them without dealing with them.

In [8]:
#locate all "and" in eu_occ
print(eu_occ[eu_occ['Occupation'].str.contains(' and ')])
#drop all "and" in eu_occ
eu_occ = eu_occ[~eu_occ['Occupation'].str.contains(' and ')]

                                             Occupation
1345                     heating equipment and supplies
1346  heating equipment and supplies distribution ma...
2755             telecommunications equipment and parts
2756  telecommunications equipment and parts distrib...
2786            textile semi-finished and raw materials


In [9]:
#save the data
eu_occ.to_csv("eu_occupations_cleaned.csv", index=False)

As before for the ONET dataset, I count the most frequent word occuring in the occupations titles, and filter through them with Chat GPT to create "general" occupations. The list will be merged later with my other data.

In [10]:
#most frequent words in EU dataset
eu_occ = pd.read_csv("eu_occupations_cleaned.csv")
from collections import Counter
all_words_eu = ' '.join(eu_occ['Occupation']).split()
word_counts_eu = Counter(all_words_eu)
most_common_words_eu = word_counts_eu.most_common(1000)
print(most_common_words_eu)
np.savetxt("most_common_words_eu.csv", most_common_words_eu, delimiter=",", fmt='%s')

[('manager', 365), ('operator', 297), ('technician', 206), ('engineer', 150), ('machine', 99), ('officer', 99), ('worker', 89), ('supervisor', 85), ('teacher', 73), ('seller', 56), ('assistant', 53), ('inspector', 53), ('ict', 47), ('production', 45), ('shop', 44), ('maker', 44), ('specialised', 43), ('engineering', 43), ('equipment', 40), ('leather', 39), ('assembler', 38), ('lecturer', 38), ('goods', 37), ('distribution', 36), ('school', 35), ('designer', 35), ('analyst', 34), ('textile', 31), ('instructor', 30), ('quality', 30), ('social', 30), ('consultant', 29), ('coordinator', 29), ('director', 28), ('construction', 27), ('specialist', 26), ('animal', 25), ('plant', 24), ('vehicle', 24), ('footwear', 24), ('secondary', 23), ('developer', 23), ('attendant', 22), ('aquaculture', 22), ('industrial', 21), ('service', 20), ('aircraft', 20), ('products', 20), ('energy', 20), ('metal', 20), ('policy', 19), ('maintenance', 19), ('product', 19), ('scientist', 18), ('vocational', 18), ('dr

In [11]:
occupations_eu = [
    "manager","operator","technician","engineer","officer","worker","supervisor","teacher",
    "seller","assistant","inspector","maker","assembler","lecturer","designer","analyst",
    "instructor","consultant","coordinator","director","specialist","developer","attendant",
    "installer","driver","therapist","artist","administrator","clerk","agent","drafter","tester",
    "planner","chief","cleaner","painter","electrician","advisor","pilot","counsellor","grader",
    "representative","broker","adviser","breeder","mechanic","trader","animator","leader",
    "trainer","fitter","researcher","buyer","dispatcher","guard","technologist","builder",
    "journalist","patternmaker","surveyor","chemist","psychologist","nurse","handler","chef",
    "auditor","operative","repairer","machinist","cutter","finisher","cook","welder","upholsterer",
    "firefighter","receptionist","engraver","polisher","guide","pharmacist","underwriter",
    "physiotherapist","executive","chiropractor","biologist","restorer","assessor","audiologist",
    "butcher","metallurgist","conductor","registrar","diver","lawyer","investigator","curator",
    "radiographer","moulder","labourer","veterinarian","hairdresser","slaughterer","porter",
    "vendor","manufacturer","writer","merchandiser","programmer","detective","setter","secretary",
    "groomer","nutritionist","osteopath","captain","teller","treasurer","bartender","librarian",
    "publisher","carpenter","weaver","maintainer","scaffolder","correctional officer","miller",
    "observer","examiner","commissioner","stewardess","stylist","farmer","appraiser",
    "superintendent","educator","performer","miner","interpreter","judge","mentor","driller",
    "volunteer","modeller","actor","acupuncturist","copywriter","aesthetician","agronomist",
    "behaviourist","hydrotherapist","anthropologist","arboriculturist","archaeologist",
    "archivist","aromatherapist","artisan","assayer","astrologer","astronaut","astronomer", 'police officer'
]


---
## [**Kaggle Job Description Dataset**](https://www.kaggle.com/datasets/ravindrasinghrana/job-description-dataset)

On paper, this is a huge dataset which contains salary data, qualifications and such. However, I only need the unique job titles, of which there are very few: around 147. In hindsight, this dataset might not be that useful, but in any case, the data cleaning function introduced during the EU dataset is applied here as well

In [12]:
#Kaggle - Occupation Titles
kaggle_occ = pd.read_csv('job_descriptions.csv')
print(f"There are {len(kaggle_occ)} occupations in the Kaggle dataset, pre cleaning.")
kaggle_occ = kaggle_occ[['Job Title']].drop_duplicates().reset_index(drop=True)
kaggle_occ.columns = ['Occupation']
print(f"There are {len(kaggle_occ)} occupations in the Kaggle dataset, post cleaning.")


There are 1615940 occupations in the Kaggle dataset, pre cleaning.
There are 147 occupations in the Kaggle dataset, post cleaning.


In [13]:
kaggle_occ_cleaned = preprocess_occupation(kaggle_occ)

#look for "and" in kaggle_occ
print(f"The following occupations contain 'and':")
print(kaggle_occ_cleaned[kaggle_occ_cleaned['Occupation'].str.contains(' and ')])

#drop all "and" in kaggle_occ
kaggle_occ_cleaned = kaggle_occ_cleaned[~kaggle_occ_cleaned['Occupation'].str.contains(' and ')]

#save the data
kaggle_occ_cleaned.to_csv("kaggle_occupations_cleaned.csv", index=False)

The following occupations contain 'and':
Empty DataFrame
Columns: [Occupation]
Index: []


As before, we can count the most common words that occur, and create a general list. ChatGPT is asked to help go through the list and take out all that are not strictly jobs.

In [14]:
#Count most frequent words in Kaggle dataset
all_words = ' '.join(kaggle_occ_cleaned['Occupation']).split()
word_counts = Counter(all_words)
most_common_words = word_counts.most_common(1000)
print(most_common_words)

np.savetxt("most_common_words_kaggle.csv", most_common_words, delimiter=",", fmt='%s')

occupations_kaggle = [
    "manager","analyst","engineer","specialist","designer","developer","coordinator","assistant","administrator","planner","nurse","director",
    "architect","representative","consultant","advisor","therapist","practitioner","executive","teacher","writer","scientist","accountant","copywriter",
    "lawyer","hygienist","physician","pediatrician","paralegal","counselor","researcher","veterinarian"
]

[('manager', 21), ('analyst', 16), ('engineer', 14), ('specialist', 10), ('designer', 9), ('developer', 7), ('marketing', 7), ('coordinator', 6), ('assistant', 5), ('network', 5), ('sales', 5), ('account', 4), ('customer', 4), ('data', 4), ('administrator', 4), ('planner', 4), ('nurse', 4), ('financial', 4), ('legal', 4), ('software', 4), ('director', 3), ('architect', 3), ('representative', 3), ('consultant', 3), ('event', 3), ('advisor', 3), ('hr', 3), ('investment', 3), ('it', 3), ('research', 3), ('therapist', 3), ('procurement', 3), ('social', 3), ('systems', 3), ('executive', 2), ('art', 2), ('teacher', 2), ('brand', 2), ('business', 2), ('chemical', 2), ('writer', 2), ('service', 2), ('support', 2), ('scientist', 2), ('database', 2), ('electrical', 2), ('environmental', 2), ('family', 2), ('practitioner', 2), ('front-end', 2), ('landscape', 2), ('market', 2), ('mechanical', 2), ('product', 2), ('project', 2), ('qa', 2), ('seo', 2), ('media', 2), ('supply', 2), ('chain', 2), ('we

In [15]:
# US Bureau of Labor Statistics - Occupational Employment and Wage Statistics (OEWS) - 2024
us_bls_occ = pd.read_excel('all_data_M_2024.xlsx', usecols=['OCC_CODE', 'OCC_TITLE'])
print(len(us_bls_occ))

414437


---
## [**US Labor Statistics (May 2024, all data)** ](https://www.bls.gov/oes/tables.htm) 

This dataset is also huge with many fields that include SOC codes for each job title. This would be very useful for categorisation of jobs, but for the time being, this is put to the side. I will again apply the preprocessing function introduced during the treatment of the eu dataset.

After this was done, some problems were not fixed. This dataset being huge (around 1,700 entries), there are many special cases I could not think of at first that needed solving. Addition functions are introduced.

Apart from this, the same process is repeated here as for the previous datasets: we form a generalised list.

In [16]:
us_bls_occ_copy = us_bls_occ[['OCC_TITLE']].copy()
us_bls_occ_copy.columns = ['Occupation']
us_bls_occ_copy = preprocess_occupation(us_bls_occ_copy)

def split_and_jobs(text):
    match = re.match(r'(.+?)\s+and\s+(.+)$', text)
    if not match:
        return [text]

    # Extract parts
    first_part = match.group(1).strip()
    second_part = match.group(2).strip()

    # split at and
    first_words = first_part.split()
    second_words = second_part.split()

    # Case 1: shared suffix after "and" (e.g., "food and beverage servers")
    if len(second_words) > 1 and second_words[-1] not in first_part.split():
        suffix = " ".join(second_words[1:])
        return [f"{first_words[0]} {suffix}", second_part]

    # Case 2: first part has multi-word prefix (e.g., "automotive technicians and repairers")
    if len(second_words) == 1:
        prefix = " ".join(first_words[:-1])
        second_job = (prefix + " " + second_words[0]).strip()
        return [first_part, second_job]

    # default case: just split into two occupations
    return [first_part, second_part]

# Expand occupations
us_bls_occ_expanded = (us_bls_occ_copy['Occupation'].apply(split_and_jobs).explode().reset_index(drop=True).to_frame())

#drop all ands in us_bls_occ
us_bls_occ_expanded = us_bls_occ_expanded[~us_bls_occ_expanded['Occupation'].str.contains(' and ')]

#save the data
us_bls_occ_expanded.to_csv("us_bls_occupations_cleaned.csv", index=False)

#most frequent words in US BLS dataset
from collections import Counter
all_words_us = ' '.join(us_bls_occ_expanded['Occupation']).split()
word_counts_us = Counter(all_words_us)
most_common_words_us = word_counts_us.most_common(1000)
print(most_common_words_us)
np.savetxt("most_common_words_us.csv", most_common_words_us, delimiter=",", fmt='%s')

occupations_us = [
    "workers","operators","technicians","teachers","managers","repairers","specialists","clerks",
    "supervisors","installers","setters","engineers","tenders","assistants","scientists",
    "attendants","mechanics","inspectors","drivers","designers","therapists","architects",
    "counselors","drafters","cleaners","instructors","therapists","practitioners",
    "collectors","judges","advisors","tapers","artists","programmers","investigators",
    "representatives","surgeons","cooks","psychologists","planners","cutters","patternmakers",
    "directors","performers","pilots","helpers","salespersons","reporters","promoters",
    "physicians","fabricators","pipefitters","auditors","officials","trainers","appraisers",
    "cosmetologists","carpenters","blockmasons","cashiers","jockeys","laborers","guards","curators",
    "solderers","detectives","dispatchers","distributors","dressmakers","drillers","pumpers",
    "sculptors","fitness trainers","painters","lawyers","nurses","developers","writers","actors",
    "producers","servicers","casters","mediators","archivists","athletes","competitors","porters",
    "bailiffs","barbers","captains","brickmasons","butchers","chemists","executives","adjusters",
    "coaches","foresters","dentists","embalmers","paramedics","entertainers","geoscientists",
    "hydrologists","maids","educators","farmworkers","filers","firefighters","wardens","cooks",
    "hairdressers","registrars","magistrate","librarians","logisticians","superintendents",
    "woodworkers","musicians","journalists","orderlies","paperhangers","plasterers","printing workers",
    "processors","brokers","singers","testers","stonemasons","tailors","guides","umpires","welders",
    "biologists","accountants","actuaries","acupuncturists","adjudicators","anesthesiologists",
    "breeders","anthropologists","auditors","audiologists","authors","tellers","bakers","bartenders",
    "chefs","chiropractors","copilots","coremakers","jailers","couriers","crane operators","dancers",
    "hygienists","dermatologists","publishers","sonographers","dietitians","dishwashers","electricians",
    "elevator operators","engravers","epidemiologists","fallers","contractors","farmers","erectors",
    "firefighters","servers","fundraisers","dealers","glaziers","gynecologists","janitors","jewelers",
    "locksmiths","lifeguards","manicurists","mathematicians","microbiologists","millwrights","molders",
    "morticians","projectionists","mechanics","composers","neurologists","anesthetists","midwives",
    "nutritionists","obstetricians","ophthalmologists","opticians","optometrists","orthodontists",
    "orthopedic surgeons","orthotists","paralegals","pediatricians","pedicurists","aides","handlers",
    "pharmacists","phlebotomists","photogrammetrists","photographers","physicians","physicists",
    "plumbers","podiatrists","carriers","postmasters","pressers","probation officers",
    "procurement officers","proofreaders","assessors","prosthetists","prosthodontists","psychiatrists",
    "telecommunicators","radiologists","conductors","yardmasters","ranchers","receptionists",
    "referees","registered nurses","rehabilitation specialists","restaurant workers","riggers",
    "roofers","roustabouts","sailors","monitors","scouts","securities officers","sewer workers",
    "sewing machine operators"
]



[('workers', 227), ('operators', 123), ('machine', 87), ('technicians', 83), ('teachers', 74), ('occupations', 57), ('miscellaneous', 54), ('equipment', 50), ('setters,', 50), ('managers', 48), ('repairers', 48), ('specialists', 46), ('clerks', 44), ('of', 43), ('service', 42), ('supervisors', 39), ('installers', 38), ('except', 36), ('operators,', 36), ('engineers', 31), ('plastic', 30), ('tenders', 28), ('assistants', 27), ('scientists', 26), ('first-line', 26), ('sales', 25), ('other', 25), ('technologists', 23), ('health', 22), ('education', 21), ('medical', 20), ('science', 20), ('food', 20), ('engineering', 19), ('social', 19), ('computer', 19), ('agents', 18), ('attendants', 17), ('transportation', 16), ('officers', 15), ('mechanics', 15), ('metal', 14), ('financial', 14), ('support', 13), ('analysts', 13), ('tool', 13), ('construction', 13), ('assemblers', 12), ('counselors', 12), ('electrical', 12), ('gambling', 12), ('personal', 12), ('aides', 12), ('agricultural', 11), ('sci

C:\Users\andra\AppData\Local\Temp\ipykernel_11068\249252778.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Occupation'] = data['Occupation'].apply(


---
## [**US Census Data (2018 Census Occupation Index)**](https://www.census.gov/topics/employment/industry-occupation/guidance/indexes.html)

This dataset is the largest of all with about 31,000 entries. it contains SOC codes and job titles and other fields that are not importante here. We only need the text. We apply our preprocessing that we did before.

In [17]:
#US census data
census_loc = 'Alphabetical-Index-of-Occupations-December-2019_Final.xlsx'
occupations = pd.read_excel(census_loc, skiprows=6)
occupations.columns = ['occupation_name', 'industry_restriction', 'occupation_code', 'SOC_code']
#print(len(occupations))
occupations = occupations[['occupation_name']]
occupations.columns = ['Occupation']
occupations = preprocess_occupation(occupations)
#print(len(occupations))


c:\Users\andra\OneDrive\Desktop\MA1_2025-2026\Applied_data_analysis\project\ada-2025-project-adacore42\_Other\andras_analysis\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\andra\AppData\Local\Temp\ipykernel_11068\249252778.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Occupation'] = data['Occupation'].apply(


We also apply the same splitting of word with "and" that we did previously for the US data. This dataset is much more complex as the previous ones, so even more treatment is needed.
1. Some jobs are related to other jobs, and the writers of the dataset in this case say (example) chicken farmer, See farmer. I drop all these lines as they are in the end same as other entries already present in the set.
2. There are entries of the form: ceo (chief executive officer). In this case I decide to create two jobs: one before the bracket, the other within the bracket
3. Some other conventions in the dataset have some symbols, and short word in some titles. These are all dropped including what comes after them


In [18]:
# Expand occupations
occupations_expanded = (occupations['Occupation'].apply(split_and_jobs).explode().reset_index(drop=True).to_frame())

#drop all ands in occupations_expanded
occupations_expanded = occupations_expanded[~occupations_expanded['Occupation'].str.contains(' and ')]

#more cleaning: 
# See point 1 above
def filter_complicated_titles(df):

    pattern = r'.+\sSee\s+"[^"]+"'  # any text followed by 'See "..."'
    mask = df['Occupation'].str.contains(pattern, na=False, case=False, regex=True)
    filtered_df = df[~mask].reset_index(drop=True)
    return filtered_df

# See point 2 above
def extract_bracketed(text):

    # Extract bracketed text
    match = re.search(r"\[([^\]]+)\]", text) # Try to match square brackets first
    if not match:
        match = re.search(r"\(([^\)]+)\)", text) # If none, try round parentheses
    
    if match:
        alternative_name = match.group(1)
    else:
        alternative_name = None
    
    # Remove bracketed text from original
    cleaned_text = re.sub(r"\[.*?\]|\(.*?\)", "", text).strip()
    return cleaned_text, alternative_name

# See point 3 above
def simplify_occupation(text):
    text = str(text).lower()  # lowercase
    # Patterns to cut off extra explanations
    cut_patterns = [
        r"\\.*",            # everything after backslash
        r"see.*",           # everything after 'see'
        r"code by.*",       # everything after 'code by'
        r"specified.*",     # everything after 'specified'
        r"as ns.*",         # everything after 'as ns'
        r"any other.*",     # everything after 'any other'
        r"\/.*"              # everything after forward slash
    ]
    
    for pattern in cut_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Apply filtering
occupations_expanded = filter_complicated_titles(occupations_expanded)
# Apply extraction of bracketed text
occupations_expanded[['Occupation', 'Alternative_Name']] = (occupations_expanded['Occupation'].apply(lambda x: pd.Series(extract_bracketed(x))))
# Apply simplification
occupations_expanded['Occupation'] = occupations_expanded['Occupation'].apply(simplify_occupation)

# Remove duplicates again
occupations_expanded = occupations_expanded.drop_duplicates(subset=['Occupation'], ignore_index=True)

While we created a new column called alternative name for the same job, we had a change of heart: identical jobs will be turned into "new" jobs. This is only done for the time being, and it might make finding synonyms more complicated later.

In [19]:
# Combine original and alternative names
alt_occupations = occupations_expanded['Alternative_Name'].dropna().reset_index(drop=True)
alt_occupations_df = pd.DataFrame(alt_occupations, columns=['Occupation'])
final = pd.concat([occupations_expanded[['Occupation']], alt_occupations_df], ignore_index=True)

#remove NaN
final = final.dropna(subset=['Occupation']).reset_index(drop=True)

# Remove empty or whitespace-only entries
final = final[final['Occupation'].str.strip().astype(bool)].reset_index(drop=True)
final = final[final['Occupation'].notna()]
final = final[final['Occupation'].str.strip().astype(bool)]
#save final occupations
final.to_csv("census_occupations_cleaned.csv", index=False)


In the case of this dataset, I did not count most common occurences, as it would have taken a long time to filter through it. However, this could be done later.

---
## Combining all data

We can finally combine all lists and occupation titles we have created so far! This will be the basis of of my list that will be used to identify jobs.

The final dataset has 33,700 entries roughly.

In [20]:
#combine the four lists made above into one final list (set removes duplicates)
final_occupations = list(set(occupations_eu + occupations_kaggle + occupations_us + occupations_1))

#drop duplicates (drop duplicates anyway out of paranoia)
final_occupations = list(set(final_occupations))

#load the cleaned data
occupations1 = pd.read_csv("occupations1.csv")
eu_occ = pd.read_csv("eu_occupations_cleaned.csv")
kaggle_occ = pd.read_csv("kaggle_occupations_cleaned.csv")
us_bls_occ = pd.read_csv("us_bls_occupations_cleaned.csv")
us_census_occ = pd.read_csv("census_occupations_cleaned.csv")

#combine all cleaned data + final_occupations into one dataframe
combined_occupations = pd.DataFrame({'Occupation': final_occupations})
combined_occupations = pd.concat([combined_occupations, eu_occ, kaggle_occ, us_bls_occ, us_census_occ, occupations1], ignore_index=True)

#drop duplicates again
combined_occupations = combined_occupations.drop_duplicates(subset=['Occupation'], ignore_index=True).dropna(subset = ['Occupation']).reset_index(drop=True)
print(len(combined_occupations))

#save the final combined occupations
combined_occupations.to_csv("final_combined_occupations.csv", index=False)


33623


---
## Enriching data

I decide to create a synonyms column which will contain the occupation and the plural version of the occupation by using an inflection engine.

This part is not necessary due to the lemmatisation of the captions data, but it is still done just in case I wnat to use the actual captions and not the tokenised version. 

In [21]:
import pandas as pd
import inflect

df = pd.read_csv("final_combined_occupations.csv")
p = inflect.engine()

def normalise_occupation(row):
    occ = row['Occupation']
    if not isinstance(occ, str) or not occ.strip():
        return row  # skip blanks

    # Ensure Synonyms is a list
    if not isinstance(row['Synonyms'], list):
        row['Synonyms'] = [row['Synonyms']] if pd.notna(row['Synonyms']) else []

    words = occ.split()
    last = words[-1]
    singular_last = p.singular_noun(last) or last
    plural_last = p.plural(singular_last)

    singular_phrase = ' '.join(words[:-1] + [singular_last])
    plural_phrase = ' '.join(words[:-1] + [plural_last])

    row['Occupation'] = singular_phrase.lower().strip()

    plural_phrase = plural_phrase.lower().strip()
    if plural_phrase not in row['Synonyms']:
        row['Synonyms'].append(plural_phrase)

    return row


# Initialize Synonyms column
df['Synonyms'] = df['Occupation'].apply(lambda x: [x.lower().strip()] if isinstance(x, str) else [])
#
#  Apply normalization
df = df.apply(normalise_occupation, axis=1)

when going through the initial application of the dataset, there are some shortcomings that I notice. There are still some entries which are not actually occupations: this is due to bad treatment of data and will hopefully be fixed later on. For the time being, the code below contains manual fix.

In [22]:
# List of occupations to remove
to_remove = [
    'brother', 'sister', 'processor', 'gutter', 'sewer', 'sight', 'lot', 'ga', 'private','nipper', 'gas', 
    'office', 'performance', 'web', 'wash', 'grip', 'gem', 'smasher', 'family', 'star sewer', 'trailer', 
    'wall', 'drywall', 'second', 'soil', 'plant', 'watch', 'bill', 'billing', 'audio', 'spotter', 'helper', 
    'footer', 'infantry','command', 'hiker', 'drive', 'set', 'server network', 'network', 'urban', 'ceiling', 
    'training', 'cut', 'community', 'social', 'drawing', 'tie', 'worker', 'hand', 'travel', 'computer', 'bridge', 'fire',
    'closer', 'subway', 'camera', 'financial', 'chair', 'ordinary', 'monitor', 'marriage', 'medium', 'potato', 
    'video', 'carry', 'fish', 'loan', 'guide', 'philosophy', 'prior', 'clock', 'heavy', 'fabric', 'reader', 'planner', 'keeper',
    'rover', 'rubber', 'counter', 'river', 'medical', 'killer', 'farm', 'porter', 'compensation', 'tool', 'mate', 'passer',
    'commercial', 'whip', 'rougher', 'entertainment', 'cotton', 'smoker', 'recreation', 'tile', 'extra', 'insurance', 'sale',
    'extra', 'master', 'feed', 'plastic', 'engine', 'refuse', 'page', 'breakfast', 'fishing', 'forest', 'signal', 'general', 'information',
    'comic'
    ]
# Remove rows with occupations in the to_remove list
df = df[~df['Occupation'].isin(to_remove)].reset_index(drop=True)

#drop duplicates based on Occupation
df = df.drop_duplicates(subset=['Occupation'], ignore_index=True)

#add "sales" as a job synonym to "salesperson"
sales_idx = df[df['Occupation'] == 'salesperson'].index
if not sales_idx.empty:
    idx = sales_idx[0]
    if 'sales' not in df.at[idx, 'Synonyms']:
        df.at[idx, 'Synonyms'].append('sales')
#save the data
df.to_csv("final_combined_occupations.csv", index=False)

---
## Synonyms (abandoned for now)

I actually attempted to use a wordnet to produce synonyms for some jobs and it works great at getting colloquial terms. However, it often introduced terms that are not actually jobs. Instead of manually going through the list, I decided to not use this in my analysis for the time being.

In [23]:
#expand using synonyms from wordnet
def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word, pos=wn.NOUN):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))
    #synonims as list
    return list(synonyms)

# take the dataset and make a new column with synonyms
combined_occupations['Synonyms'] = combined_occupations['Occupation'].apply(get_synonyms)

#print(combined_occupations)

#save the data
combined_occupations.to_csv("final_combined_occupations_with_synonyms.csv", index=False)

This code will take the occupations, and add them to the synonyms list. It will make all occupations singular and plural term are moved into synonyms if they are in occupation column.

In [24]:
import pandas as pd
import inflect
import ast

p = inflect.engine()

def normalise_occupation(row):
    occ = row['Occupation']
    if not isinstance(occ, str) or not occ.strip():
        return row  # skip blanks

    words = occ.split()

    # Handle last word (the noun)
    last = words[-1]
    singular_last = p.singular_noun(last) or last
    plural_last = p.plural(singular_last)

    # Construct phrases
    singular_phrase = ' '.join(words[:-1] + [singular_last])
    plural_phrase = ' '.join(words[:-1] + [plural_last])

    # Update occupation → singular
    row['Occupation'] = singular_phrase.lower().strip()

    # Add plural to synonyms if not already there
    if plural_phrase.lower().strip() not in row['Synonyms']:
        row['Synonyms'].append(plural_phrase.lower().strip())

    return row

df = pd.read_csv("final_combined_occupations_with_synonyms.csv")

# Convert Synonyms column from string to list
df['Synonyms'] = df['Synonyms'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
df = df.apply(normalise_occupation, axis=1)



In [25]:
#remove duplicates again
df = df.drop_duplicates(subset=['Occupation'], ignore_index=True)

This code would find the plural of the all synonyms to the jobs that I found before. Again, this is not used for the time being.

In [26]:
def normalize_synonyms(synonyms):
    normalised = set()
    
    # Flatten if nested
    if isinstance(synonyms, (list, set, tuple)):
        flat_synonyms = []
        for s in synonyms:
            if isinstance(s, (list, set, tuple)):
                flat_synonyms.extend(s)
            else:
                flat_synonyms.append(s)
    else:
        flat_synonyms = [synonyms]
    
    for syn in flat_synonyms:
        if not isinstance(syn, str) or not syn.strip():
            continue
        
        words = syn.strip().split()
        if not words:
            continue

        # Singularize and pluralize only the last word
        last_word = words[-1]
        singular_last = p.singular_noun(last_word) or last_word
        plural_last = p.plural(singular_last)

        singular_phrase = ' '.join(words[:-1] + [singular_last])
        plural_phrase = ' '.join(words[:-1] + [plural_last])

        normalised.add(singular_phrase.lower())
        normalised.add(plural_phrase.lower())

    return sorted(normalised)

# Apply to your DataFrame
df['Synonyms'] = df['Synonyms'].apply(normalize_synonyms)

df.to_csv("final_combined_occupations_with_synonyms.csv", index=False)

The code below contains manual fixes hat I could find as issues when testing. 

In [27]:
# List of occupations to remove
to_remove = [
    'brother', 'sister', 'processor', 'gutter', 'sewer', 'sight', 'lot', 'ga', 'private','nipper', 'gas', 
    'office', 'performance', 'web', 'wash', 'grip', 'gem', 'smasher', 'family', 'star sewer', 'trailer', 
    'wall', 'drywall', 'second', 'soil', 'plant', 'watch', 'bill', 'billing', 'audio', 'spotter', 'helper', 'footer', 'infantry'
    'command', 'hiker', 'drive', 'set', 'server network', 'network',
]

# Remove all in one go
df = df[~df['Occupation'].isin(to_remove)].reset_index(drop=True)

#turn synonyms into a list of strings
import ast

#convert "assemblyman or woman" to "assemblyman" and "assemblywoman" in separate rows
df['Occupation'] = df['Occupation'].apply(
    lambda x: ['assemblyman', 'assemblywoman'] if x == 'assemblyman or woman' else [x]
)
df = df.explode('Occupation').reset_index(drop=True)

df.loc[df['Occupation'] == 'assemblyman', 'Synonyms'] = "['assemblyman', 'assemblymen']"
df.loc[df['Occupation'] == 'assemblywoman', 'Synonyms'] = "['assemblywoman', 'assemblywomen']"


# Convert each string to a Python list
df['Synonyms'] = df['Synonyms'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

#if something starts with "and" remove "and " and keep the rest
def remove_and_beginning(occupation):
    if occupation.lower().startswith('and '):
        return occupation[4:].strip()
    return occupation
df['Occupation'] = df['Occupation'].apply(remove_and_beginning)

#charwoman has as synonym "woman", "women", "char" and "chars" - remove them
df.loc[df['Occupation'] == 'charwoman', 'Synonyms'] = df.loc[df['Occupation'] == 'charwoman', 'Synonyms'].apply(
    lambda syns: [s for s in syns if s not in ['woman', 'women', 'char', 'chars']]
)

#valet has "man", "men", gentleman", "gentlemen" as synonyms - remove them
df.loc[df['Occupation'] == 'valet', 'Synonyms'] = df.loc[df['Occupation'] == 'valet', 'Synonyms'].apply(
    lambda syns: [s for s in syns if s not in ['man', 'men', 'gentleman', 'gentlemen', 'gentleman\'s gentleman', 'gentleman\'s gentlemans', 'gentlemans']]
)
#correct u s to us in synonyms. it's in the synonyms of "u s postal inspection officer"
# Locate the row by occupation
df[df['Occupation'] == 'u s postal inspection officer'] = 'us postal inspection officer'
df.loc[df['Occupation'] == 'us postal inspection officer', 'Synonyms'] = df.loc[df['Occupation'] == 'us postal inspection officer', 'Synonyms'].apply(
    lambda syns: [s.replace('u s', 'us') for s in syns])

C:\Users\andra\AppData\Local\Temp\ipykernel_11068\46995080.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'us postal inspection officer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[df['Occupation'] == 'u s postal inspection officer'] = 'us postal inspection officer'


Saving data.

In [28]:
# Convert each string to a Python list
df['Synonyms'] = df['Synonyms'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df.to_csv("final_combined_occupations_with_synonyms.csv", index=False)